## Input data requirements

The input ice sheet model should be saved as a netCDF file. There are several requirements for the comparison:

### `Lithk` variable

The uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](http://www.climate-cryosphere.org/wiki/index.php?title=ISMIP6_wiki_page). 


In [ ]:
# set geospatial python3 kernel to provide needed packages
import os,sys
import glob as glob
import numpy as np
import h5py
import xarray as xr
import glob as glob


import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
# from matplotlib import rc
# rc('mathtext', default='regular')

sys.path.insert(0, os.path.abspath('..'))


# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')

In [ ]:
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# Load the dataset(model data)
nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_GIS_JPL_ISSM_asmb.nc'
gis_ds = xr.open_dataset(nc_filename)
lithk = gis_ds['lithk']

# Define start and end dates
start_date = np.datetime64('2000-01-01')
end_date = np.datetime64('2015-01-01')

# Interpolate lithk values at the start and end dates
lithk_start = lithk.interp(time=start_date).data.transpose().flatten()
lithk_end = lithk.interp(time=end_date).data.transpose().flatten()

# Calculate the difference
lithk_delta = lithk_end - lithk_start

# Replace NaN values with 0
lithk_delta[np.isnan(lithk_delta)] = 0


# Now lithk_delta is ready for further analysis or plotting

x_coords = gis_ds['x'].values
y_coords = gis_ds['y'].values
x_resolution = abs(x_coords[1] - x_coords[0])
y_resolution = abs(y_coords[1] - y_coords[0])

# Ice thickness (m) to mass (kg) to gigatonnes(Gt)
# ice thickness*area* density of ice* 1e-9
rho_ice = 934 # (density of ice, kg/m^3)
# lithk_delta = (lithk_delta * x_resolution*y_resolution)*rho_ice * 1e-9
lithk_delta = (lithk_delta * x_resolution*y_resolution)*rho_ice * 1e-12

# Create a list of Point geometries from coordinate grids
points = [Point(x, y) for x in x_coords for y in y_coords]

# Flatten lithk_delta to match the points list if it isn't already flattened
lithk_delta_flat = lithk_delta.flatten()

# Create DataFrame
lithk_df = pd.DataFrame({
    'geometry': points,
    'lithk_delta': lithk_delta_flat
})

# Convert DataFrame to GeoDataFrame
lithk_gdf = gpd.GeoDataFrame(lithk_df, geometry='geometry', crs="EPSG:3413")



shapefile_path = "/home/jovyan/CmCt/notebooks/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
# Load basin shapefile 
basins_gdf = gpd.read_file(shapefile_path)
# ensure it's in the same CRS
# basins_gdf = basins_gdf.to_crs("EPSG:3413")

# Perform spatial join
joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", op='intersects')


# Sum lithk_delta values by basin
basin_mass_change_sums = joined_gdf.groupby('index_right')['lithk_delta'].sum()
print('Basin_mass_change_sums =', basin_mass_change_sums)


# Sum all of the basin mass change
model_total_mass_balance= basin_mass_change_sums.sum()
print('Total_mass_change_sum =', model_total_mass_balance)


In [ ]:
import matplotlib.pyplot as plt

# Plot polygons first
fig, ax = plt.subplots(figsize=(10, 10))
basins_gdf.plot(ax=ax, color='lightblue', edgecolor='black')

# Overlay points with color representing lithk_delta
joined_gdf.plot(ax=ax, column='lithk_delta', cmap='viridis', markersize=0.5, alpha=0.7, legend=True)

plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Load the CSV file
csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_greenland_2021_Gt.csv'
mass_balance_data = pd.read_csv(csv_file_path)

# Adjust the column names based on the CSV content
date_column = 'Year'
mass_balance_column = 'Mass balance (Gt/yr)'

# Convert the 'Year' column to datetime format
mass_balance_data[date_column] = pd.to_datetime(mass_balance_data[date_column], format='%Y')

# Filter the data between the start and end dates
start_date = np.datetime64('2000-01-01')
end_date = np.datetime64('2015-01-01')
filtered_data = mass_balance_data[(mass_balance_data[date_column] >= start_date) & 
                                  (mass_balance_data[date_column] <= end_date)].copy()

# Convert Mass balance from Gt/yr to Gt/month
filtered_data.loc[:, 'Mass_balance_Gt'] = filtered_data[mass_balance_column] / 12

# Sum the total mass balance in the filtered range
imbie_total_mass_change_sum = filtered_data['Mass_balance_Gt'].sum()

# Print the results
print(f"Total Mass Balance from {start_date} to {end_date}: {imbie_total_mass_change_sum} Gt")




In [ ]:
#Difference of imbie-model  mass change for Greenland
delta_masschange=imbie_total_mass_change_sum-model_total_mass_balance
print(f"Difference of imbie-model  mass change: {delta_masschange}")